# VL5 – Dynamische Datenvisualisierung / Motion Design (Python)

**Kontext:** Übungen zu *Animation*, *Motion Design* und *Storytelling mit Daten*  
**Tools:** Plotly (interaktiv + animiert), pynimate (GIF/Video‑Animationen)

## Lernziele

Nach der Bearbeitung dieses Notebooks kannst du …

1. Daten für Animationen **tidy** vorbereiten (Filtern, Pivotieren, Aggregieren, Missing Values behandeln),
2. mit **Plotly Express** animierte Scatterplots („Rosling Bubble Chart“) und Animationseinstellungen (frame duration, transition) steuern,
3. mit **pynimate** eine Animation als **GIF** exportieren (Canvas, Plot‑Objekte, FPS/Interval),
4. Motion‑Design‑Entscheidungen **begründet** treffen (Tempo, Easing/Übergänge, visuelle Hierarchie, Reduktion),
5. die Aussage einer Visualisierung kritisch reflektieren (Skalen, Kodierungen, Vergleichbarkeit über Zeit).

## Arbeitsweise im Notebook

- **Erst lauffähig machen, dann verschönern:** Wenn du Fehler bekommst, kommentiere #optional‑Blöcke zunächst aus.
- **Mini‑Iterationen:** Nach jedem Schritt kurz prüfen (DataFrame anzeigen, `fig.show()`/GIF preview).
- **Dokumentiere Entscheidungen:** Beantworte die *Transferfragen* direkt unter den Übungen.

> Dont worry, Übung macht den Meister! 🚀


In [ ]:
# Setup: Bibliotheken installieren (nur nötig, wenn sie noch nicht installiert sind)
#optional: In lokalen Umgebungen/Colab ausführen – in manchen Uni‑Setups ist Installation gesperrt.
# %pip install plotly pandas pynimate geopandas matplotlib pillow

In [ ]:
# Imports

# Datenhandling
import pandas as pd
import numpy as np

# Interaktive/animierte Visualisierung
import plotly.express as px

# Animation als GIF/Video (pynimate)
import pynimate as nim
from pynimate import Canvas, Barplot
from pynimate.utils import human_readable

# I/O & Utilities
import os
import json

# Matplotlib (wird von pynimate genutzt)
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
from matplotlib.animation import PillowWriter

In [ ]:
# Daten laden und vorbereiten (INKAR)
# Hinweis: Pfad an deine Umgebung anpassen (z. B. relativer Pfad im Projektordner).

inkar_path = r"C:\Users\Matthias\Documents\Ohm_Vorlesung\project\notebooks\data\inkar_bayern_nordbayern.parquet"

# Daten aus dem github: https://github.com/MAD1982/dataviz_WS2526/raw/main/inkar/inkar_bayern_nordbayern.parquet

# load parquet
df_inkar = pd.read_parquet(inkar_path)

# Quick sanity check: existiert die Filterspalte?
df_inkar["Nordbayern"].value_counts()


In [ ]:
# Datenexploration (kurz, aber wichtig!)
# Ziel: verstehen, welche Spalten/Keys du für Filter, Join und Animation brauchst.

df_inkar.info()
df_inkar.head()

# Welche Indikatoren sind verfügbar?
df_inkar["Indikator"].drop_duplicates().sort_values()

# Kennziffern/Keys: wichtig für Joins (z. B. mit GeoJSON) und für eindeutige Gruppen in Animationen
df_inkar[["Name", "Kennziffer"]].drop_duplicates().sort_values("Kennziffer")

# Filter: Großstädte (Beispiel aus der Vorlesung)
by_grosse_staedte = [
    "09161000", "09162000", "09362000",
    "09562000", "09563000", "09564000",
    "09663000", "09761000"
]

df_inkar = df_inkar[df_inkar["Kennziffer"].isin(by_grosse_staedte)]
df_inkar["Name"].drop_duplicates().sort_values()


## Übung 1: Scatterplot Bubble Chart (Rosling‑Style)

### Thema
**Entwicklung von Wohlstand (BIP/Einwohner) und Arbeitslosenquote** in den Großstädten Bayerns seit 1998.

### Lernziele (Übung 1)
- Du kannst ein **Long‑Format** in ein **Wide‑Format** pivotieren, um mehrere Indikatoren als Spalten zu bekommen.
- Du kannst eine **animierte Scatterplot‑Story** aufbauen: Zustände pro Jahr, gleiche Punkte über Zeit („animation_group“).
- Du kannst **Skalen & Kodierungen** (x/y/size/color) kritisch prüfen.

### Didaktische Hinweise
- Starte mit **Datenprüfung**: Sind beide Indikatoren pro Stadt/Jahr vorhanden? (Missing Values!)
- Achte auf **Einheiten** (EUR vs. %) und auf eine sinnvolle **Skalierung** (ggf. log‑Skala für BIP).
- Motion‑Design‑Regel: **Nicht alles animieren** – erst klare Botschaft, dann Bewegung als Verstärker.

### Aufgabe
1. Filtere auf die Indikatoren **„Bruttoinlandsprodukt je Einwohner“** und **„Arbeitslosenquote“**.
2. Pivotiere die Daten so, dass du pro Stadt & Jahr beide Indikatoren als **separate Spalten** hast.
3. Erstelle einen animierten Scatterplot:
   - x: BIP/Einwohner  
   - y: Arbeitslosenquote  
   - frame: Jahr (`Zeitbezug`)  
   - group: Stadtname (`Name`)

### Transferfragen (kurz beantworten)
- Was verändert sich an deiner Interpretation, wenn du **size** statt über BIP über **Bevölkerung** kodierst?
- Welche Risiken entstehen durch **Farben pro Stadt** (viele Kategorien)? Welche Alternative wäre besser?
- Würdest du für eine Präsentation eher **Animation** oder **Small multiples** nutzen – und warum?


In [ ]:
# ÜBUNG 1 – Daten vorbereiten + animierter Rosling‑Scatter (Plotly)
# Ziel: Long -> Wide (Pivot) und dann als Animation über Jahre erzählen.
# Tipp: Wenn etwas nicht funktioniert: zuerst df_wide anzeigen und prüfen, ob die Spalten existieren.

indikatoren_liste = ["Arbeitslosenquote", "Bruttoinlandsprodukt je Einwohner"]
df_inkar = df_inkar[df_inkar["Indikator"].isin(indikatoren_liste)]

# Your Code Here

NameError: name 'df_inkar' is not defined

### Bonus (Übung 1): Motion‑Tuning

- Passe die **Geschwindigkeit** der Animation an (frame duration / transition).
- Kodieren der Punktgröße über **Einwohnerzahl**: Prüfe, ob die Größenunterschiede lesbar sind (ggf. `size_max`).

**Transfer:** Wann wirkt schnelle Animation „cool“, aber inhaltlich schlechter? Nenne ein Beispiel.


In [ ]:
# ÜBUNG 1 – Bonus: Animationstempo & Größenkodierung

# Geschwindigkeit anpassen
speed = 100  # Geschwindigkeit in Millisekunden pro Frame

# Your Code Here

## Übung 2: Bar Chart Race (pynimate)

### Thema
**Hausärzte je 1000 Einwohner** in bayerischen Landkreisen über die Zeit.

### Lernziele (Übung 2)
- Du kannst Zeitreihendaten so aggregieren, dass sie für ein **Bar Chart Race** geeignet sind (Top‑k pro Jahr).
- Du kannst in pynimate mit **Canvas + Barplot** eine Animation erzeugen und exportieren.
- Du kannst Lesbarkeit sichern: konsistente Achse, sinnvolle `top_n`, ruhige Beschriftung.

### Didaktische Hinweise
- Baue zuerst ein **statisches Ranking** für ein Jahr → erst dann animieren.
- Prüfe, ob „Top 25“ über viele Jahre **flackert** (wechselnde Kreise) – das kann die Botschaft schwächen.
- Motion‑Design: Nutze Bewegung, um **Dynamik** zu zeigen – aber halte Achse & Layout stabil.

### Aufgabe
Erstelle ein Bar Chart Race, das **pro Jahr nur die Top 25** Landkreise mit der besten Versorgung (Hausärzte je 1000 Einwohner) zeigt.

> Hinweis: Lade die INKAR‑Daten für diese Übung gern nochmal frisch und filtere dafür neu.


In [ ]:
# ÜBUNG 2 – Bar Chart Race (pynimate)
# Ziel: Zeitreihen in 'long' -> pro Jahr Top‑k -> Animation.

# Daten laden
df_inkar = pd.read_parquet(inkar_path)

# Nordbayern filtern
df_inkar = df_inkar[df_inkar["Nordbayern"] == True]

# Nur Kreise
df_inkar = df_inkar[df_inkar["Raumbezug"] == "Kreise"]

# Daten filtern
indikatoren_liste = ["Hausärzte"]
df_inkar = df_inkar[df_inkar["Indikator"].isin(indikatoren_liste)]

# Your Code Here

### Bonus (Übung 2): Small multiples (statisch) statt Animation

Erstelle eine **Small‑multiples‑Ansicht** (z. B. mehrere Balkendiagramme nebeneinander) für die **Top 10** im aktuellsten verfügbaren Jahr.

**Transfer:** Welche Fragestellung beantwortet Small multiples besser als ein Bar Chart Race?


#### Bonus (Übung 2): Top 10 – beste Versorgung pro Jahr

Reduziere auf die **Top 10** pro Jahr und beobachte:
- Wird die Animation verständlicher?
- Erkennst du *dauerhafte Gewinner* vs. *kurze Ausreißer*?


In [ ]:
# Wieviele Kreise sollen angezeigt werden
TOP_N = 10

# Daten vorbereiten
df_plot = (
    df_wide[["Zeitbezug", "Name", "Hausärzte"]]
    .rename(columns={
        "Zeitbezug": "Jahr",
        "Name": "Kreis/kreisfreie Stadt",
        "Hausärzte": "Hausärzte je 1000 Einwohner",
    })
    .copy()
)

# Your Code Here

#### Bonus (Übung 2): Top 10 – schlechteste Versorgung pro Jahr

Analog: Zeige pro Jahr die **10 schlechtesten** Kreise.

**Transfer:** Warum kann „Bottom‑10“ politisch/kommunikativ sensibler sein als „Top‑10“?


In [ ]:
# Wieviele Kreise sollen angezeigt werden
BOTTOM_N = 10

# Daten vorbereiten (identisch zu Top-10)
df_plot = (
    df_wide[["Zeitbezug", "Name", "Hausärzte"]]
    .rename(columns={
        "Zeitbezug": "Jahr",
        "Name": "Kreis/kreisfreie Stadt",
        "Hausärzte": "Hausärzte je 1000 Einwohner",
    })
    .copy()
)

# Your Code Here

## Übung 3: Animierte (Geo‑)Heatmap

### Thema
Visualisiere den **Wanderungssaldo** (Zuzüge – Fortzüge) je Landkreis über die Zeit (Bayern).

### Lernziele (Übung 3a)
- Du kannst Tabellen‑Daten mit **Geodaten** (GeoJSON) joinen (Schlüssel, Kreiscodes).
- Du kannst in Plotly eine **animierte Choropleth/Heatmap** erzeugen (frame = Jahr).
- Du kannst Karten‑Basics reflektieren: Projektion, Farbskalen, Vergleichbarkeit.

### Didaktische Hinweise
- Prüfe zuerst den **Join‑Key**: stimmen Kreisschlüssel zwischen INKAR und GeoJSON überein?
- Achte auf **farbliche Vergleichbarkeit über Jahre** (fixe Farbskala statt autoscale).
- Motion‑Design bei Karten: Lieber ruhige Übergänge, weil sich „viel bewegt“, obwohl nur Farben wechseln.

### Aufgabe
Erstelle eine animierte Heatmap für den Wanderungssaldo in Bayern (Kreisebene) und nutze die GeoJSON‑Kreisdaten aus der Vorlesung.


In [ ]:
# Quelle: https://opendatalab.de/projects/geojson-utilities/ -> Kreisdaten für müssen zunöchst heruntergeladen werden
local_geojson = r"C:\Users\Matthias\Documents\Ohm_Vorlesung\project\data\landkreise_simplify20.geojson"

# Alternativ aus dem GitHub https://github.com/MAD1982/dataviz_WS2526/blob/main/data/gemeinden_simplify20.geojson laden 

#GeoJSON laden
if not os.path.exists(local_geojson):
    raise FileNotFoundError(f"GeoJSON nicht gefunden: {local_geojson}")

with open(local_geojson, "r", encoding="utf-8") as f:
    counties = json.load(f)

geo_feature_key = "properties.RS"
geo_ids = pd.Series([str(feat["properties"]["RS"]) for feat in counties["features"]], name="RS")

print("GeoJSON geladen.")

# INKAR Daten laden
df_inkar = pd.read_parquet(inkar_path)

# INKAR Daten filtern
indikatoren_liste = ["Gesamtwanderungssaldo"]
df_inkar = df_inkar[df_inkar["Indikator"].isin(indikatoren_liste)]
df_inkar = df_inkar[df_inkar["Raumbezug"] == "Kreise"]

# Your Code Here

### Bonus (Übung 3a): Kumulierte Entwicklung

Erstelle zusätzlich die Animation für den **kumulierten Wanderungssaldo** (aufsummiert über Jahre).

**Transferfragen**
- Welche Kreise zeigen dauerhaft Abwanderung? Sind das eher urbane oder ländliche Räume?
- Wann ist „kumuliert“ hilfreicher als „jährlich“ – und wann irreführend?


In [ ]:
# Your Code Here

## Übung 3b: Animierte Time‑Series (pynimate)

### Aufgabe
Erstelle einen animierten **Time‑Series‑Plot** für eine oder mehrere INKAR‑Variablen in Nordbayern (z. B. Ärzte, Hausärzte, Kinderärzte, Krankenhausbetten). Exportiere die Animation als **GIF**.

### Didaktische Hinweise
- Entscheide dich für **1–2 Variablen** zum Start → sonst wird es visuell überladen.
- Nutze Labels sparsam: lieber wenige, gut gesetzte Hinweise als Text‑Overkill.
- Stelle sicher, dass die Zeitachse **sortiert** ist und Missing Values behandelt sind.

### Transfer
- Welche Story erzählst du: *Trend*, *Bruch*, *Saison*, *Vergleich*?
- Wie würdest du dieselben Daten ohne Animation erzählen (Small multiples / Sparklines)?


##### Vorlage: Covid‑19 Fälle in Deutschland im Jahr 2020 (angepasst von einem pynimate‑Beispiel)

Nutze die Vorlage als Referenz für:
- Daten laden & bereinigen
- Styling (Theme)
- Canvas/Plot aufsetzen
- Animation exportieren


In [ ]:
# TEMPLATE (pynimate) – Beispiel: Covid‑19 (Referenz)
# Nutze das als Muster für Datenaufbereitung + Animationsexport.

#optional: Styling (Dark Theme) – reine Optik

for side in ["left", "right", "top", "bottom"]:
    mpl.rcParams[f"axes.spines.{side}"] = False

mpl.rcParams["figure.facecolor"] = "#001219"
mpl.rcParams["axes.facecolor"] = "#001219"
mpl.rcParams["savefig.facecolor"] = "#001219"

# 2) Data Fetching & Processing -> Johns Hopkins University CSSE COVID-19 Dataset

URL_CONFIRMED = (
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
    "csse_covid_19_data/csse_covid_19_time_series/"
    "time_series_covid19_confirmed_global.csv"
)

URL_DEATHS = (
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
    "csse_covid_19_data/csse_covid_19_time_series/"
    "time_series_covid19_deaths_global.csv"
)

URL_RECOVERED = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")


META_COLS = ["Province/State", "Country/Region", "Lat", "Long"]


def _process_jhu_series(url: str, country: str, col_name: str) -> pd.DataFrame:
    """Load a JHU global time series CSV and return a single aggregated country series."""
    df = pd.read_csv(url)

    if "Country/Region" not in df.columns:
        raise ValueError("Unexpected JHU schema: 'Country/Region' column missing.")

    df = df[df["Country/Region"] == country].copy()
    if df.empty:
        raise ValueError(f"Country '{country}' not found in dataset: {url}")

    date_cols = [c for c in df.columns if c not in META_COLS]
    s = df[date_cols].sum(axis=0)  # aggregate across provinces/rows
    s.index = pd.to_datetime(s.index, errors="raise")
    s = s.sort_index()

    return s.to_frame(name=col_name)


def get_covid_data(
    country: str = "Germany",
    start: str = "2020-03-01",
    end: str = "2021-01-01",
    clip_negative: bool = False,
) -> pd.DataFrame:
    """
    Returns daily new values (diff of cumulative) for a given country and date range.
    Output columns: cases, deaths
    Index formatted as '%Y-%m-%d' for pynimate.
    """
    confirmed = _process_jhu_series(URL_CONFIRMED, country, "cases")
    deaths = _process_jhu_series(URL_DEATHS, country, "deaths")
    recovered = _process_jhu_series(URL_RECOVERED, country, "recovered")

    df = pd.concat([confirmed, deaths, recovered], axis=1)

    # Convert cumulative -> daily new
    df = df.diff().fillna(0)

    # Optional: remove negative corrections for didactic clarity
    if clip_negative:
        df = df.clip(lower=0)

    # Filter date range
    df = df.loc[pd.to_datetime(start):pd.to_datetime(end)]

    # Pynimate expects string index matching the format passed to LineDatafier
    df.index = df.index.strftime("%Y-%m-%d")

    return df

# 3) Animation
def post(self, i):
    self.ax.yaxis.set_major_formatter(
        tick.FuncFormatter(lambda x, pos: human_readable(x))
    )

# Parameters (easy to tweak)
COUNTRY = "Germany"
START = "2020-03-01"
END = "2020-12-31"
FPS = 24
OUTFILE = "covid_cases_recovered_pynimate"

# Fetch data
df = get_covid_data(
    country=COUNTRY,
    start=START,
    end=END,
    clip_negative=False  # set True if you want strictly non-negative daily values
)

# Create animation with pynimate
cnv = nim.Canvas() # Create canvas
dfr = nim.LineDatafier(df, "%Y-%m-%d", "12h")  # Create datafier

# Create line plot
plot = nim.Lineplot(
    dfr,
    post_update=post,
    palettes=["Set3"],
    scatter_markers=False,
    legend=True,
    fixed_ylim=True,
    grid=False,
)

# Make deaths visually distinct
plot.set_column_linestyles({"recovered": "dashed"})

plot.set_title(
    f"Daily new COVID-19 cases & recovered – {COUNTRY} ({START} to {END})",
    y=1.05,
    color="w",
    weight=600,
)

# Axis labels
plot.set_xlabel("Date", color="w", size=11)

# Y-Axis label with human-readable formatting
plot.set_time(
    callback=lambda i, datafier: datafier.data.index[i].strftime("%d %b, %Y"),
    color="w",
    size=13,
)

# Y-Axis label with human-readable formatting
plot.set_line_annots(lambda col, val: f"({human_readable(val)})", color="w")
plot.set_legend(labelcolor="w")

# Running totals over the shown range (sum of daily values)
plot.set_text(
    "sum",
    callback=lambda i, datafier: ( # cumulative sums over shown range
        "Cumulative (range)\n"
        f"Cases : {human_readable(datafier.data.cases.iloc[:i+1].sum())}\n"
        f"Recovered: {human_readable(datafier.data.recovered.iloc[:i+1].sum())}"
    ),
    size=10,
    x=0.70,
    y=0.18,
    color="w",
)

plot.set_xticks(colors="w", length=0, labelsize=10)
plot.set_yticks(colors="w", labelsize=10)

# Add plot to canvas and animate
cnv.add_plot(plot) # Add plot to canvas
cnv.animate(interval=20) # milliseconds between frames

# Save animation as GIF
print("Saving animation...")
cnv.ani.save(f"{OUTFILE}.gif", writer=PillowWriter(fps=FPS))
print("Done.")


plt.show()

#### ... und jetzt du (Umsetzung)

Nutze die Template‑Struktur und ersetze:
- Datenquelle (INKAR statt Covid – oder eigene Zeitreihe),
- Titel/Labels,
- `FPS`, `DURATION`, `OUTFILE`.

Schreibe zum Schluss 3–5 Sätze: **Welche Designentscheidung hat den größten Effekt auf Verständlichkeit?**


In [ ]:
# ÜBUNG 3b – INKAR Time‑Series Animation (pynimate)
# Ziel: Zeitreihen (Nordbayern) vorbereiten und als GIF exportieren.

# Your Code Here